# IDC Tutorial: Data exploration of slide microscopy images

This notebook demonstrates how to explore slide microscopy data using the [Imaging Data Commons (IDC)](https://portal.imaging.datacommons.cancer.gov/).

Besides radiology, slide microscopy is the second major imaging modality in the IDC. Slide microscopy images show thin sections of tissue samples (e.g., from a resected tumor) at microscopic resolution. They provide a unique glimpse into cellular architecture and function that is essential for diagnosing complex diseases like cancer. Computerized analysis makes the assessment of slide microscopy images more reproducible and less time consuming and it enables the extration of novel digital biomarkers from tissue images.

This tutorial provides a **5-minute** introduction to the way slide microscopy data is organized within the IDC and demonstrates how to examine available data and build a data set for further analysis. For a more comprehensive tutorial including training of a tissue classification model on IDC-hosted slide microscopy data, see [here](https://github.com/ImagingDataCommons/idc-comppath-reproducibility).  
  
To learn more about the IDC platform, please visit the [IDC user guide](https://learn.canceridc.dev/).

If you have any questions, bug reports, or feature requests please feel free to contact us at the [IDC discussion forum](https://discourse.canceridc.dev/)!

## Prerequisites

**Authenticate:** To access IDC resources, you have to authenticate with your Google identity. Follow the link generated by the code below and enter the displayed verification code to complete the Google authentication process.

In [ ]:
from google.colab import auth
auth.authenticate_user()

**Create a Google Cloud Platform project:** In order to run this notebook you need to have a Google Cloud Platform project. You can learn how to create your own project [here](https://www.youtube.com/watch?v=i08S0KJLnyw). Billing information is not required for running this tutorial. You are still encouraged to apply for free cloud credits from IDC by submitting the application form referenced [here](https://learn.canceridc.dev/introduction/requesting-gcp-cloud-credits) and use them for other tutorials. Once you have the google project, set `my_project_id` below to the ID of your GCP project.

In [ ]:
my_project_id = 'idc-pathomics-000'

## Environment setup

Import the required Python modules.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
sns.set_theme()
import warnings
with warnings.catch_warnings(): # Hide Python warnings to improve readability.
    warnings.simplefilter('ignore')

Determine who and where we are.

In [ ]:
curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print('Current directory :', curr_dir[-1])
print('Hostname          :', curr_droid[-1])
print('Username          :', curr_pilot[-1])

## Dataset selection and exploration

IDC relies on the Google Cloud Platform (GCP) for storage and management of DICOM data. The data are contained in so-called [storage buckets](https://cloud.google.com/storage/docs/key-terms#buckets), from which they can be retrieved on a requester pays basis. Currently, all pathology whole-slide images (WSI) are located in the *idc-open* bucket.

Metadata for the DICOM files — including standard DICOM tags, but also non-DICOM metadata — are stored in the BigQuery table *canceridc-data.idc_current.dicom_all*. The IDC Documentation gives further information on [data organization](https://learn.canceridc.dev/data/organization-of-data) and [code examples](https://learn.canceridc.dev/cookbook/bigquery) on how to query the table. The easiest way to access BigQuery tables from a Jupyter notebook is to use [BigQuery cell magic](https://cloud.google.com/bigquery/docs/visualize-jupyter#querying-and-visualizing-bigquery-data) with the `%%bigquery` command.

The following statement loads relevant metadata of all slide images from the CPTAC-LUAD and CPTAC-LSCC datasets into a pandas data frame called `slides_df`.

In [ ]:
%%bigquery slides_df --project=$my_project_id

-- First, create subqueries to facilitate data selection from the dicom_all table.
-- Several DICOM attributes are renamed to be more intuitive (e.g. 'TotalPixelMatrixColumns' is renamed to 'width').
WITH
  -- The 'sm_images' subquery summarizes basic metadata of slide microscopy images (SM).
  sm_images AS (
    SELECT
      -- digital_slide_id: unique numeric identifier of a digital slide, i.e., a digital image of a physical slide.
      SeriesInstanceUID AS digital_slide_id,
      -- physical_slide_id: unique numeric identifier of the physical slide as prepared in the wet lab.
      ContainerIdentifier AS physical_slide_id,
      -- patient_id: unique numeric identifier of the patient from whose tissue the physical slide was obtained.
      PatientID AS patient_id,
      -- case_id: unique numeric identifier of the study in the context of which the ditial slide was created.
      StudyInstanceUID AS case_id,
      -- width: image width in pixels.
      TotalPixelMatrixColumns AS width,
      -- height: image height in pixels.
      TotalPixelMatrixRows AS height,
      -- collection_id: numeric or character sequence describing the dataset the physical slide is part of.
      collection_id,
      -- crdc_instance_uuid: universally unique identifier of the DICOM instance.
      crdc_instance_uuid,
      -- gcs_url: GCS URL needed to access the DICOM file
      gcs_url,
      -- pixel_spacing: image pixel spacing in mm/px
      CAST(SharedFunctionalGroupsSequence[SAFE_OFFSET(0)].
              PixelMeasuresSequence[SAFE_OFFSET(0)].
              PixelSpacing[SAFE_OFFSET(0)] AS FLOAT64) AS pixel_spacing,
      -- compression: compression type
      CASE TransferSyntaxUID
          WHEN '1.2.840.10008.1.2.4.50' THEN 'jpeg'
          WHEN '1.2.840.10008.1.2.4.91' THEN 'jpeg2000'
          ELSE 'other'
      END AS compression,
    FROM
      -- we are using version 11 of the IDC data
      bigquery-public-data.idc_v11.dicom_all
    WHERE
      -- limit query to slide microscopy images (SM) excluding non-volume images like label images
      Modality = 'SM' AND ImageType[SAFE_OFFSET(2)] = 'VOLUME'
  ),
  -- The 'tissue_types' subquery indicates whether a slides contains normal, tumor or other tisse.
  tissue_types AS (
    SELECT DISTINCT *
    FROM (
      SELECT
        SeriesInstanceUID AS digital_slide_id,
        CASE steps_unnested2.CodeValue
            WHEN '17621005' THEN 'normal' -- meaning: 'Normal' (i.e., non neoplastic)
            WHEN '86049000' THEN 'tumor' -- meaning: 'Neoplasm, Primary'
            ELSE 'other' -- meaning: 'Neoplasm, Metastatic'
        END AS tissue_type
      FROM
        bigquery-public-data.idc_v11.dicom_all
        CROSS JOIN
          UNNEST (SpecimenDescriptionSequence[SAFE_OFFSET(0)].PrimaryAnatomicStructureSequence) AS steps_unnested1
        CROSS JOIN
          UNNEST (steps_unnested1.PrimaryAnatomicStructureModifierSequence) AS steps_unnested2
    )
  )


-- Select all DICOM files from the CPTAC-LUAD / CPTAC-LUSC datasets that are JPEG/JPEG2000 compressed and
-- contain normal or tumor tissue. We do not need to restrict the query to frozen tissue sections since all
-- slides in CPTAC show frozen tissue. The results are ordered by their crdc_instance_uuid to ensure
-- deterministic output order which is important for reproducibility.
SELECT
  a.*,
  b.tissue_type,
  (REPLACE (REPLACE(collection_id, 'cptac_luad', 'luad'), 'cptac_lscc', 'lscc')) AS cancer_subtype
FROM
  sm_images AS a
  JOIN tissue_types AS b ON b.digital_slide_id=a.digital_slide_id
WHERE
  (a.collection_id = 'cptac_luad' OR a.collection_id = 'cptac_lscc')
  AND a.compression != 'other'
  AND (b.tissue_type = 'normal' OR b.tissue_type = 'tumor')
ORDER BY crdc_instance_uuid

We reduce the obtained data frame to the images that are digitized at 5x magnification (corresponding to a pixel spacing between 0.0019 and 0.0021 mm).

In [ ]:
slides_df.query('pixel_spacing > 0.0019 & pixel_spacing < 0.0021', inplace=True)

With standard [pandas](https://pandas.pydata.org/) functionality, we can easily validate and summarize the compiled metadata.

In [ ]:
# Assert uniqueness of slide_id values
assert(slides_df.digital_slide_id.is_unique)

# Assert validity of class labels
assert set(slides_df.tissue_type.unique()) == set(['normal', 'tumor'])
assert set(slides_df.cancer_subtype.unique()) == set(['luad', 'lscc'])

display(slides_df.head())
print('Total number of slides: ', len(slides_df))
nr_slides = slides_df.groupby('cancer_subtype').size()
nr_patients = slides_df.drop_duplicates('patient_id').groupby('cancer_subtype').size()
print('--> %d slides from %d LUAD patients' % (nr_slides['luad'], nr_patients['luad']))
print('--> %d slides from %d LSCC patients' % (nr_slides['lscc'], nr_patients['lscc']))

Using standard [matplotlib](https://matplotlib.org/) functionality, we can easily visualize some aspects of interest. The following code produces two histograms. The left graph shows the frequencies of numbers of slides per patient, while the right graph shows the proportions of slides derived from healthy or tumor tissue.

In [ ]:
fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Number of slides per patient
slides_per_patient = slides_df.groupby(['patient_id']).size()
plot1 = sns.histplot(data=slides_per_patient, discrete=True, ax=ax1, shrink=0.9)
ax1.update({'xlabel': 'Number of slides', 'ylabel': 'Number of patients'})
ax1.xaxis.set_major_locator(MaxNLocator(integer=True)) # Force integer labels on x-axis

# Distribution of tissue types
plot2 = sns.histplot(data=slides_df, x='cancer_subtype', hue='tissue_type', multiple='stack', palette = ['C1', 'C2'], ax=ax2, shrink=0.7)
ax2.update({'xlabel': 'Cancer subtype', 'ylabel': 'Number of slides'})
legend = plot2.get_legend()
legend.set_title('Tissue type')
legend.set_bbox_to_anchor((1, 1))

Any case with its slides can also be viewed and explored in detail using the IDC viewer.

In [ ]:
def get_idc_viewer_url(study_UID):
    return "https://viewer.imaging.datacommons.cancer.gov/slim/studies/" + study_UID

print(get_idc_viewer_url(slides_df['case_id'].iloc[0]))
print(get_idc_viewer_url(slides_df['case_id'].iloc[100]))

Finally, we can save the information as CSV file *slides_metadata.csv* to be used later for further analysis such as downloading the cohort and training plus evaluation of a tissue classification model as outlined in [these tutorials](https://github.com/ImagingDataCommons/idc-comppath-reproducibility).  

In [ ]:
slides_df.to_csv('./slides_metadata.csv', index=False)